# L2 正则化（Ridge / Weight Decay）

## 核心思想

L2 正则化通过在损失函数中添加权重平方和的惩罚项，限制权重的大小，防止过拟合。

## 数学表达

```
Loss_total = Loss_original + λ × Σwᵢ²
```

其中 λ 是正则化强度超参数。

## L1 vs L2 对比

| 特性 | L1 正则化 | L2 正则化 |
|------|-----------|----------|
| 惩罚项 | Σ\|wᵢ\| | Σwᵢ² |
| 权重分布 | 稀疏（多个0） | 小而分散 |
| 特征选择 | 是 | 否 |
| 计算效率 | 较低 | 较高 |
| 常用场景 | 特征选择 | 防止过拟合 |

In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt

# 设置随机种子
tf.random.set_seed(42)
np.random.seed(42)

print(f"TensorFlow 版本: {tf.__version__}")

## 1. 基本用法

In [ ]:
# L2 正则化层
# l2(0.001) 表示正则化系数 λ = 0.001
layer_l2 = keras.layers.Dense(
    units=100,
    activation='elu',
    kernel_regularizer=keras.regularizers.l2(0.001),
    kernel_initializer='he_normal'
)

print("L2 正则化层创建成功")
print(f"正则化系数: 0.001")

## 2. 完整训练示例

In [ ]:
# 加载数据
(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.fashion_mnist.load_data()
X_valid, X_train = X_train_full[:5000] / 255.0, X_train_full[5000:] / 255.0
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]
X_test = X_test / 255.0

print(f"训练集: {X_train.shape}")
print(f"验证集: {X_valid.shape}")

In [ ]:
def create_model(l2_lambda=0.0):
    """
    创建带可配置 L2 正则化的模型
    
    Parameters:
    -----------
    l2_lambda : float
        L2 正则化系数，0 表示不使用正则化
    
    Returns:
    --------
    keras.Model
        编译好的模型
    """
    regularizer = keras.regularizers.l2(l2_lambda) if l2_lambda > 0 else None
    
    model = keras.models.Sequential([
        keras.layers.Flatten(input_shape=(28, 28)),
        keras.layers.Dense(256, activation='elu', 
                          kernel_regularizer=regularizer,
                          kernel_initializer='he_normal'),
        keras.layers.Dense(128, activation='elu',
                          kernel_regularizer=regularizer,
                          kernel_initializer='he_normal'),
        keras.layers.Dense(64, activation='elu',
                          kernel_regularizer=regularizer,
                          kernel_initializer='he_normal'),
        keras.layers.Dense(10, activation='softmax')
    ])
    
    model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    
    return model

# 创建对比模型
model_no_reg = create_model(l2_lambda=0.0)
model_with_l2 = create_model(l2_lambda=0.001)

print("模型创建完成")

In [ ]:
# 训练对比
EPOCHS = 30
BATCH_SIZE = 64

print("训练无正则化模型...")
history_no_reg = model_no_reg.fit(
    X_train, y_train,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    validation_data=(X_valid, y_valid),
    verbose=0
)
print("完成")

print("训练 L2 正则化模型...")
history_with_l2 = model_with_l2.fit(
    X_train, y_train,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    validation_data=(X_valid, y_valid),
    verbose=0
)
print("完成")

In [ ]:
# 绘制训练曲线
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 准确率
axes[0].plot(history_no_reg.history['accuracy'], 'b-', label='无正则化 (训练)')
axes[0].plot(history_no_reg.history['val_accuracy'], 'b--', label='无正则化 (验证)')
axes[0].plot(history_with_l2.history['accuracy'], 'r-', label='L2 正则化 (训练)')
axes[0].plot(history_with_l2.history['val_accuracy'], 'r--', label='L2 正则化 (验证)')
axes[0].set_title('准确率对比')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Accuracy')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# 损失
axes[1].plot(history_no_reg.history['loss'], 'b-', label='无正则化 (训练)')
axes[1].plot(history_no_reg.history['val_loss'], 'b--', label='无正则化 (验证)')
axes[1].plot(history_with_l2.history['loss'], 'r-', label='L2 正则化 (训练)')
axes[1].plot(history_with_l2.history['val_loss'], 'r--', label='L2 正则化 (验证)')
axes[1].set_title('损失对比')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Loss')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('l2_regularization_comparison.png', dpi=150, bbox_inches='tight')
plt.show()

# 测试集评估
print("\n测试集评估:")
test_loss_no, test_acc_no = model_no_reg.evaluate(X_test, y_test, verbose=0)
test_loss_l2, test_acc_l2 = model_with_l2.evaluate(X_test, y_test, verbose=0)

print(f"无正则化: 准确率 = {test_acc_no:.4f}")
print(f"L2 正则化: 准确率 = {test_acc_l2:.4f}")

## 3. 可视化权重分布

In [ ]:
# 对比权重分布
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 无正则化模型的权重
weights_no_reg = model_no_reg.layers[1].get_weights()[0].flatten()
axes[0].hist(weights_no_reg, bins=50, density=True, alpha=0.7, color='blue')
axes[0].set_title(f'无正则化 - 权重分布\n均值={weights_no_reg.mean():.4f}, 标准差={weights_no_reg.std():.4f}')
axes[0].set_xlabel('权重值')
axes[0].set_ylabel('密度')
axes[0].grid(True, alpha=0.3)

# L2 正则化模型的权重
weights_l2 = model_with_l2.layers[1].get_weights()[0].flatten()
axes[1].hist(weights_l2, bins=50, density=True, alpha=0.7, color='red')
axes[1].set_title(f'L2 正则化 - 权重分布\n均值={weights_l2.mean():.4f}, 标准差={weights_l2.std():.4f}')
axes[1].set_xlabel('权重值')
axes[1].set_ylabel('密度')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\n权重统计:")
print(f"无正则化 - 权重范数: {np.linalg.norm(weights_no_reg):.4f}")
print(f"L2 正则化 - 权重范数: {np.linalg.norm(weights_l2):.4f}")

In [ ]:
# 验证代码正确性
print("L2 正则化模块测试完成")
print("\n关键要点:")
print("1. L2 正则化限制权重大小，产生小而分散的权重")
print("2. 常用系数范围: 0.0001 - 0.01")
print("3. L2 是最常用的正则化方法之一")